# Knowledge Distillation in MHIST Dataset

In [1]:
import os
import sys
import cv2
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Utilities import *

In [2]:
def load_images_from_folder(folder):
    images = []
    file_names = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            file_names.append(filename)
    return images, file_names

path = "mhist_dataset/images"
data, file_names = load_images_from_folder(path)
CSVfile = "mhist_dataset/annotations.csv"

labels = pd.read_csv(CSVfile, usecols = [1])
Partitions = pd.read_csv(CSVfile, usecols = [3])
labels = labels.to_numpy()
Partitions = Partitions.to_numpy()
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(len(data)):
    if Partitions[i] == 'train':
        X_train.append(data[i])
        if (labels[i] == 'SSA'):
            y_train.append([1,0])
        if (labels[i] == 'HP'):
            y_train.append([0,1])
    if Partitions[i] == 'test':
        X_test.append(data[i])
        if (labels[i] == 'SSA'):
            y_test.append([1,0])
        if (labels[i] == 'HP'):
            y_test.append([0,1])
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2175, 224, 224, 3)
(977, 224, 224, 3)
(2175, 2)
(977, 2)


In [3]:
# Load Resnet
resNetBase= tf.keras.applications.resnet_v2.ResNet50V2(
    include_top = False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling=None,
)
for layer in resNetBase.layers:
    layer.trainable = False
x = tf.keras.layers.Flatten()(resNetBase.output)
x = tf.keras.layers.Dense(2)(x)
restNet = tf.keras.Model(inputs=resNetBase.input, outputs=x)

In [4]:
train_evaluate_resnet(restNet,X_train,X_test, y_train, y_test)

Epoch 1: Class_accuracy: 18.12%
Epoch 2: Class_accuracy: 18.12%


In [5]:
studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top = False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling=None,
)
for layer in studenModel2.layers:
    layer.trainable = False
x = tf.keras.layers.Flatten()(studenModel2.output)
x = tf.keras.layers.Dense(2)(x)
mobileNet = tf.keras.Model(inputs=studenModel2.input, outputs=x)


In [6]:
train_and_evaluate_mobileNet_using_KD(mobileNet,restNet,X_train,X_test, y_train, y_test, 0.5,4)

Epoch 1: Class_accuracy: 1.77%
Epoch 2: Class_accuracy: 1.77%
